In [1]:
!pip install pyDOE
!pip install cma


  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=9e67dec7e1d5813dea8dceca66b00f3ec7ad6e482c73fa11adcd653ce205d9ab
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.8MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import bbobbenchmarks as bn
import sklearn

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F13 '''
def F13(X):
    f = bn.F13()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data

''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))
      

''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 1300.69 ,random_state=0 , l1_ratio=0.9898, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 69.69 , C = 1000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_13_4000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv('test_13_800Samples.csv').iloc[:,-1]
path = "train_latent_200D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv('test_latent_200D.csv', index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,101):
        Cols.append(Columns[j]+'_Z'+str(i))

opt = cma.CMAOptions()
opt.set ("seed" , 0)
opt.set ("maxfevals" , 100000)

n_obs , dim =  30, 100
G = DOE(n_obs, dim)
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,300])
for i in range(G[1].shape[0]):
    print ('Run : '+ str(i))
    min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 0.1566 , options=opt) [0]
    min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 0.1566 , options=opt) [0]
    min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 0.1566 , options=opt) [0]
    X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=904329, Mon Apr 13 17:59:05 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 1.037920019810425e+04 1.0e+00 1.50e-01  2e-01  2e-01 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 17:59:05 2020)
final/bestever f-value = 1.037920e+04 1.037920e+04
incumbent solution: [0.87024859 0.83165734 0.31115476 0.63766747 0.35046423 0.5704096
 0.13714375 0.38899031 ...]
std deviations: [0.15035918 0.15035827 0.15051019 0.15041754 0.15036462 0.15038826
 0.15039994 0.15040347 ...]
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=920583, Mon Apr 13 17:59:05 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 1.037971877756385e+04 1.0e+00 1.51e-01  2e-01  2e-01 0:00.0


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [10379.2001981] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=10379.200198, sigma=1.50e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=10379.200198, sigma=1.50e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')

termination on tolfun=1e-11 (Mon Apr 13 17:59:05 2020)
final/bestever f-value = 1.037972e+04 1.037972e+04
incumbent solution: [0.82312039 0.95041089 0.16376408 0.66230952 0.50026167 0.5150665
 0.02230564 0.45608825 ...]
std deviations: [0.15088402 0.15093309 0.15090804 0.15083608 0.15084894 0.15080565
 0.15084855 0.1508406  ...]
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=914019, Mon Apr 13 17:59:05 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 1.037920019810425e+04 1.0e+00 1.51e-01  2e-01  2e-01 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 17:59:05 2020)
final/bestever f-value = 1.037920e+04 1.037920e+04
incumbent solution: [ 0.89653579  0.91561     0.2473018   0.68437877  0.48180792  0.63308158
 -0.01277592  0.48925422 ...]
std deviations: [0.15056508 0.15055468 0.15050019 0.15054466 0.15072048 0.15061045
 0.15054713 0.15054782 ...]
Run : 1
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=880935, Mon Apr 13 17:59

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [10379.2001981] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=10379.200198, sigma=1.51e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=10379.200198, sigma=1.51e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')

termination on tolfun=1e-11 (Mon Apr 13 17:59:05 2020)
final/bestever f-value = 1.037920e+04 1.037920e+04
incumbent solution: [ 0.00589167  0.4093198   0.75331222  0.37185934 -0.00726983  0.65885741
  0.9052379   0.86956182 ...]
std deviations: [0.15083133 0.15072905 0.15078725 0.15071117 0.15064484 0.15079228
 0.15078844 0.1508195  ...]
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=958954, Mon Apr 13 17:59:05 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 1.037971877756385e+04 1.0e+00 1.51e-01  2e-01  2e-01 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 17:59:06 2020)
final/bestever f-value = 1.037972e+04 1.037972e+04
incumbent solution: [0.18741354 0.46186053 0.60269473 0.45739413 0.02810006 0.54982082
 0.80961737 0.86805993 ...]
std deviations: [0.15053631 0.1505434  0.15067538 0.15051383 0.15053239 0.15065421
 0.15059624 0.15063135 ...]
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=824154, Mon Apr 13 17:59:06 202

...

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [10379.71877756] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=10379.718778, sigma=1.50e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=10379.718778, sigma=1.50e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  '

(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=912056, Mon Apr 13 17:59:18 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 1.037920019810425e+04 1.0e+00 1.51e-01  2e-01  2e-01 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 17:59:18 2020)
final/bestever f-value = 1.037920e+04 1.037920e+04
incumbent solution: [-0.03378172  0.71886796  0.80318713  1.06410724  0.21011575  0.23509935
  0.35722576  0.47434538 ...]
std deviations: [0.1504538  0.15056783 0.15042775 0.15050832 0.15046131 0.15052822
 0.15052901 0.15058569 ...]
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=951823, Mon Apr 13 17:59:18 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 1.037971877756385e+04 1.0e+00 1.51e-01  2e-01  2e-01 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 17:59:19 2020)
final/bestever f-value = 1.037972e+04 1.037972e+04
incumbent solution: [0.08847085 0.88055401 0.75055614 1.02426058 0.36293251 0.

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [10379.2001981] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=10379.200198, sigma=1.51e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=10379.200198, sigma=1.51e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')

(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=885706, Mon Apr 13 17:59:19 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 1.037920019810425e+04 1.0e+00 1.50e-01  2e-01  2e-01 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 17:59:19 2020)
final/bestever f-value = 1.037920e+04 1.037920e+04
incumbent solution: [-0.01411661  0.77956646  0.7232512   0.89279011  0.25250668  0.30588152
  0.45344463  0.58967205 ...]
std deviations: [0.15040283 0.15032624 0.15040708 0.15049219 0.15043132 0.15050544
 0.15053662 0.1504477  ...]


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [10379.2001981] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=10379.200198, sigma=1.50e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=10379.200198, sigma=1.50e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('F13_200_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F13(X_Values.iloc[i,:100])
    SVM_Fun [i] = F13(X_Values.iloc[i,100:200])
    ELN_Fun [i] = F13(X_Values.iloc[i,200:300])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([4948.58503964]), count=array([1])) 87.35385451767698
SVM
ModeResult(mode=array([4936.43568066]), count=array([1])) 89.1262091920703
ELN
ModeResult(mode=array([4995.52713695]), count=array([1])) 84.5940726962341
